# Clean up

**Note:** Please set kernel to `Python 3 (Data Science)`

---


## Introduction

This notebook is used to delete the relevant IAM roles, policy, RedShift cluster, Glue database and secret in Secret Manager.

Note: This notebook does not delete VPC, SageMaker Studio, SageMaker Pipelines, Code*, S3, EFS etc. You can delete the SageMaker project with the AWS CLI command `aws sagemaker delete-project --project-name X`. This will remove the MLOps components like CodePipeline. 

Before deleting the CloudFormation, the following components needs to be deleted manually:
- In SageMaker Studio, shutdown SageMaker Studio by going to `File` -> `Shutdown` -> `Shut down all`
- EFS

After around 5 minutes, you can delete the CloudFormation stack. Some possible issues and solutions:
- If the CloudFormation has issues deleting the VPC, you can do so manually. 

---

### Variables
Variable name for secret in Secret Manager. RedShift, Athena and Glue information are stored in the secret.

In [ ]:
secret_name='bankdm_redshift_login' 


### Import the necessary libraries and create client session


In [ ]:
from botocore.exceptions import ClientError
import json
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')
glue = boto3.client('glue')
iam = boto3.client('iam')
sts = boto3.client('sts')
accountID = sts.get_caller_identity()["Account"]  

session = boto3.session.Session()
region = session.region_name

lambda_client = boto3.client('lambda')

Load information from secret

In [ ]:
try:
    get_secret_value_response = secretsmanager.get_secret_value(
            SecretId=secret_name
        )
    secret_arn=get_secret_value_response['ARN']

except ClientError as e:
    print("Error retrieving secret. Error: " + e.response['Error']['Message'])
    
else:
    # Depending on whether the secret is a string or binary, one of these fields will be populated.
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
    else:
        secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
secret_json = json.loads(secret)
redshift_cluster_identifier = secret_json['dbClusterIdentifier']

database_name_glue = secret_json['database_name_glue']


Delete RedShift cluster

In [ ]:
try:
    response = redshift.delete_cluster(
        ClusterIdentifier=redshift_cluster_identifier,
        SkipFinalClusterSnapshot=True
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ClusterNotFound':
        print("Cluster does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Delete Glue database

In [ ]:
try:
    response = glue.delete_database(
        Name=database_name_glue
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityNotFoundException':
        print("Glue database does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Delete secret

In [ ]:
try:
    response = secretsmanager.delete_secret(
        SecretId=secret_name,
        ForceDeleteWithoutRecovery=True
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        print("Secret does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

### Delete IAM roles and policies

In [ ]:
role='BankDM-RedShift'

# Detach AWS policy
policyname = ['SecretsManagerReadWrite', 'AmazonRedshiftFullAccess', 'AmazonSageMakerFullAccess', 'AmazonS3FullAccess', 'AmazonAthenaFullAccess']
for i in range(len(policyname)):
    policy = f'arn:aws:iam::aws:policy/{policyname[i]}'
    print(policy)
    try:
        response = iam.detach_role_policy(
            RoleName=role,
            PolicyArn=policy
        )
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("IAM policy not attached to the role. This is ok.")
        else:
            print("Unexpected error: %s" % e)

# Delete role
try:
    response = iam.delete_role(
        RoleName=role,
    )

except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntityException':
        print("Role does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)


### Delete Lambda function

In [ ]:
try:
    response = lambda_client.delete_function(
            FunctionName='bankdm-redshift-dl',
        )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        print("Lambda function not found. This is ok")
    else:
        print("Unexpected error: %s" % e) 

### Delete Lambda role

In [ ]:
role='BankDM-Lambda'

# Detach AWS policy
policyname = ['SecretsManagerReadWrite', 'AmazonRedshiftFullAccess', 'AmazonSageMakerFullAccess', 'AmazonS3FullAccess']
for i in range(len(policyname)):
    policy = f'arn:aws:iam::aws:policy/{policyname[i]}'
    print(policy)
    try:
        response = iam.detach_role_policy(
            RoleName=role,
            PolicyArn=policy
        )
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("IAM policy not attached to the role. This is ok.")
        else:
            print("Unexpected error: %s" % e)

# Detach service role
policyname = ['AWSLambdaBasicExecutionRole']
for i in range(len(policyname)):
    policy = f'arn:aws:iam::aws:policy/service-role/{policyname[i]}'
    print(policy)
    try:
        response = iam.detach_role_policy(
            RoleName=role,
            PolicyArn=policy
        )
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("IAM policy not attached to the role. This is ok.")
        else:
            print("Unexpected error: %s" % e)

            
# Delete role
try:
    response = iam.delete_role(
        RoleName=role,
    )

except ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchEntityException':
        print("Role does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)
